In [1]:
import potcorr
import psutil
import os
import lab
import numpy as np
import pickle
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from scipy.sparse import csr_matrix
import sys
#from pympler import asizeof

In [2]:
cell = lab.bn_12x12
# cell = lab.mos2_tt
#cell = lab.bn_6x6
print('Starting job for '+cell['prefix']+'...')

#Eps0 =  read_eps.Epsmat(cell['folder']+'chi0mat.h5')
#Eps1 = read_eps.Epsmat(cell['folder']+'chimat.h5')
#np.save(cell['folder']+'chi0mat.npy', Eps1.mat)

ttkw=potcorr.PotCorr(cell)
print(ttkw.prefix)
print(ttkw.lattpara_unit)
# print(type(mos2.lattpara_unit))

ttkw.fft_init()

ttkw.epsmat_init_interp(G_vec2ind_dict=cell['folder_G_info']+'G_vec2ind.pkl',
                        G_ind2vec_dict=cell['folder_G_info']+'G_ind2vec.pkl')
#print(ttkw.g_tuple_floor_tt)
#print(ttkw.g_rhoind_tt)
#print(ttkw.g_tuple_q_tt)

with open(cell['folder_model']+'model_all_re.pkl','rb') as file:
    model_all_re = pickle.load(file)
with open(cell['folder_model']+'model_all_im.pkl','rb') as file:
    model_all_im = pickle.load(file)
with open(cell['folder_G_info']+'Gind_list.pkl','rb') as file:
    Gind_list = pickle.load(file)




Starting job for bn...
bn
[ 2.51  2.51 25.  ]
Begin FFT grids Initialization
FFT grids:   216    216    180
0 / 216
1 / 216
2 / 216
3 / 216
4 / 216
5 / 216
6 / 216
7 / 216
8 / 216
9 / 216
10 / 216
11 / 216
12 / 216
13 / 216
14 / 216
15 / 216
16 / 216
17 / 216
18 / 216
19 / 216
20 / 216
21 / 216
22 / 216
23 / 216
24 / 216
25 / 216
26 / 216
27 / 216
28 / 216
29 / 216
30 / 216
31 / 216
32 / 216
33 / 216
34 / 216
35 / 216
36 / 216
37 / 216
38 / 216
39 / 216
40 / 216
41 / 216
42 / 216
43 / 216
44 / 216
45 / 216
46 / 216
47 / 216
48 / 216
49 / 216
50 / 216
51 / 216
52 / 216
53 / 216
54 / 216
55 / 216
56 / 216
57 / 216
58 / 216
59 / 216
60 / 216
61 / 216
62 / 216
63 / 216
64 / 216
65 / 216
66 / 216
67 / 216
68 / 216
69 / 216
70 / 216
71 / 216
72 / 216
73 / 216
74 / 216
75 / 216
76 / 216
77 / 216
78 / 216
79 / 216
80 / 216
81 / 216
82 / 216
83 / 216
84 / 216
85 / 216
86 / 216
87 / 216
88 / 216
89 / 216
90 / 216
91 / 216
92 / 216
93 / 216
94 / 216
95 / 216
96 / 216
97 / 216
98 / 216
99 / 216
10

In [6]:
flatten_gind = ttkw.g_rhoind_tt.flatten()
flatten_qvec = ttkw.g_tuple_q_tt.flatten()

In [7]:
val_size = len(np.where(flatten_gind < len(Gind_list))[0])

In [12]:
chi_values = np.zeros((val_size*len([0,1]), 1),dtype=complex)
chi_indices_j = np.zeros(val_size*len([0,1]),dtype=int)
chi_indices_i = np.zeros(val_size*len([0,1]),dtype=int)
start_idx = 0
for i in [0,1]:
    g_qvec_ind_i=np.where(flatten_gind==i)[0]
    for j in range(len(Gind_list)):
        print(j)
        g_qvec_ind_j=np.where(flatten_gind==j)[0]
        end_idx = start_idx+len(g_qvec_ind_j)
        qinp_set = np.array([np.array([a[0],a[1]]) for a in flatten_qvec[g_qvec_ind_j]])
        #print(flatten_qvec[g_qvec_ind])
        g0_,g1_ = i,j
        chi_re_set = model_all_re[f'model_{g0_}_{g1_}'].predict(qinp_set)
        chi_im_set = model_all_im[f'model_{g0_}_{g1_}'].predict(qinp_set)
        chi_set = chi_re_set + 1j*chi_im_set
        chi_values[start_idx:end_idx] = chi_set
        chi_indices_j[start_idx:end_idx] = g_qvec_ind_j
        chi_indices_i[start_idx:end_idx] = g_qvec_ind_i
        start_idx = end_idx
        #print(chi_set)
        #print(g_qvec_ind)
#print(chi_values)
#print([x for x in chi_indices])
    
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121


In [20]:
len(chi_indices)

26352

In [22]:
sparse_vector = csr_matrix((chi_values.flatten(),(np.zeros_like(chi_indices), chi_indices)),shape=(1, len(flatten_gind)))

In [51]:
np.shape(np.random.rand(500,2))

(500, 2)

In [50]:
np.shape(qinp_set)

(144, 2)